In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# if using mac
df = pd.read_excel('/Users/zhanghantang/PycharmProjects/sucide_project/BIOM40forUSC.xlsx')
data = df[df['SI'].notnull()]
y = data['SI']
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'CFIS']

FileNotFoundError: [Errno 2] No such file or directory: '/Users/zhanghantang/PycharmProjects/sucide_project/BIOM40forUSC.xlsx'

In [3]:
# windows
df = pd.read_excel('E:\PHD\sucide_project\BIOM40forUSC.xlsx')
data = df[df['SI'].notnull()]
y = data['SI']
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'CFIS']

In [4]:
'''
data processing issues：
1. column 'CFI-S.PheneVisit' data type is string, i am not clear its internal meaning and how to convert to float data type
2. after 1 Psychiatric, data has some missing value

divide into 2 categories
Category 1: High Risk

Those with suicidal ideation (having thoughts of suicide) and those with a suicide plan (having a plan to carry out suicide) can be grouped into a high-risk category as they are actively considering and planning suicide.
Category 2: Past Attempts/ Hospitalizations

Those with a history of suicide attempt(s) and those with hospitalization(s) for suicidality can be grouped into a second category as they have a history of suicidal behavior but are not currently at an elevated risk of suicide.
'''
# column 'CFI-S.PheneVisit' data type is string, i am not clear its internal meaning and how to convert to float data type
x = x.drop(labels='CFI-S.PheneVisit', axis=1)
# drop these column directly
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'RAB3GAP2Biom240234_at']
new_y = []
for i in y:
    if i == 0 or i ==1:
        new_y.append(0)
    else:
        new_y.append(1)

In [8]:
# train test split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
X = StandardScaler().fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size=0.33, random_state=10)
print(len(X_train), len(X_test))


505 249


In [9]:
# tune parameter
from sklearn.model_selection import GridSearchCV
model = LR()
# 不同的solver可能只支持不同的有限的penalty
param_grid = {'penalty':['l1', 'l2', 'elasticnet'], 'C':[0.001, 0.01, 0.1, 1, 10, 100],
              'solver':['liblinear', 'sag', 'sega']}
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X, new_y)
print("Best penalty: ", grid_search.best_params_['penalty'])
print("Best C: ", grid_search.best_params_['C'])
print("solver: ", grid_search.best_params_['solver'])
print("Best score: ", grid_search.best_score_)


d:\python\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\python\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\python\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\python\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\python\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\python\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\python\lib\site-pac

Best penalty:  l1
Best C:  0.001
solver:  liblinear
Best score:  0.8103487858719646


d:\python\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\python\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\python\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\python\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\python\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\python\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\python\lib\site-pac

In [11]:
best_model = LR(C=0.01, penalty='l1', solver='liblinear')
best_model.fit(X, new_y)
best_model.score(X, new_y)

0.8103448275862069

In [13]:
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.8032128514056225